## ChatGPT

This is just some messing around.

Docs:

* how to upload images: https://platform.openai.com/docs/guides/vision
* types of models: https://stackoverflow.com/questions/75774873/openai-api-error-this-is-a-chat-model-and-not-supported-in-the-v1-completions
* token limits: https://platform.openai.com/settings/organization/limits
* models: https://platform.openai.com/docs/models/model-endpoint-compatibility

In [1]:
#!pip install openai

In [2]:
dir_api = '/Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/'

In [13]:
import openai
import base64
from openai import OpenAI
from PIL import Image
import numpy as np
import json
import re
import pickle
import os

In [18]:
def load_image(image_path, fac=0.5):
    img = Image.open(image_path).convert('RGB')
    new_size = np.round(np.array(img.size)*fac).astype('int')
    img = img.resize(new_size, Image.Resampling.LANCZOS)
    #img = np.array(img)
    #with open(image_path, "rb") as image_file:
    img.save('/Users/jnaiman/Downloads/tmp/tmp_img.png')
    with open('/Users/jnaiman/Downloads/tmp/tmp_img.png','rb') as image_file:
        #return base64.b64encode(img).decode("utf-8")
        return base64.b64encode(image_file.read()).decode("utf-8")

In [5]:
# parsing
def parse_qa(level_parse, plot_level, qa, j, types):
    keys_tmp = list(j[level_parse][plot_level].keys())
    keys = []
    for k in keys_tmp:
        if '(' in k:
            k = k.split('(')[0].rstrip()
        keys.append(k)
    
    keys = np.unique(keys).tolist()
    
    for k in keys:
        v = ''
        kk = ''
        for t in types:
            if k + " " + t in j[level_parse][plot_level]:
                v = j[level_parse][plot_level][k + " " + t]
                #kk = k + " " + t
                break
        if v == '':
            v = j[level_parse][plot_level][k]
            #kk = k
        if 'A' in v: # no plot
            if type(v['A']) == type({}):
                ans = list(v['A'].values())[0]
            else:
                ans = v['A']
            if type(v['Q']) == type({}):
                que = list(v['Q'].values())[0]
            else:
                que = v['Q']
        else: # plotX
            for kk,vv in v.items():
                ans = vv['A']
                while type(ans) == type({}):
                    ans_1 = list(ans.values())[0]
                    if 'plot' in list(ans.keys())[0]:
                        ans = ans_1
                        break
                    ans = ans_1
            # for kk,vv in v.items():
            #     ans = vv['A']
            #     while type(ans) == type({}):
            #         ans = list(ans.values())[0]
                que = vv['Q']
        out = {'Q':que, 'A':ans, 'Level':level_parse, 'type':plot_level, 'Response':""}
        # if there is a plot
        if 'plot' in list(v.keys())[0]:
            out['plot number'] = list(v.keys())[0]
            #import sys; sys.exit()
        qa.append(out)
    return qa


In [6]:
# setup
with open('/Users/jnaiman/.openai/key.txt','r') as f:
    api_key = f.read()

client = OpenAI(
  api_key=api_key.strip(),  # this is also the default, it can be omitted
)

In [24]:
iFile = 0
iMax = 250
verbose = False

fac = 0.5
while iFile < iMax:
    print('on', iFile, 'of', iMax)
    if os.path.exists(dir_api + 'Picture'+str(int(iFile)) + '.pickle'):
        print('have file already:', dir_api + 'Picture'+str(int(iFile)) + '.pickle')
        iFile += 1
        continue
    err = False
    # do we have it?
    try:
        image_path = '/Users/jnaiman/Downloads/data_full_v2/Picture'+str(int(iFile))+'.png'
        encoded_image = load_image(image_path, fac=fac)
        # get questions
        with open('/Users/jnaiman/Downloads/data_full_v2/Picture'+str(int(iFile))+'_qa.json','r') as f:
            j = json.loads(f.read())
            j = json.loads(j)
    except Exception as e:
        print(e)
        err = True
        pass

    if err:
        iFile += 1
        continue


    # create QA
    qa = []
    
    for k,v in j['Level 1']['Figure-level questions'].items():
        out = {'Q':v['Q'], 'A':v['A'], 'Level':'Level 1', 'type':'Figure-level questions', 'Response':""}
        qa.append(out)
    
    # what kinds?
    types = ['(words + list)', '(words)']
    
    # get uniques
    level_parse = 'Level 1'
    plot_level = 'Plot-level questions'
    qa = parse_qa(level_parse, plot_level, qa, j, types)
    
    level_parse = 'Level 2'
    plot_level = 'Plot-level questions'
    qa = parse_qa(level_parse, plot_level, qa, j, types)
    
    level_parse = 'Level 3'
    plot_level = 'Plot-level questions'
    qa = parse_qa(level_parse, plot_level, qa, j, types)


    # query chatgpt
    responses = []

    #try:
    for question_list in qa:
        success = False
        iFac = 1
        while not success:
            try:
                question = question_list['Q']
                print('   on question:',question)
                # Prepare the API request
                prompt = f"I am going to show you an image. Here is the image: [Image: {encoded_image}]. Now, {question}"
                
                # Send the request to the GPT-4o API
                response = client.chat.completions.create(
                    #model="gpt-4",
                    #model="gpt-4o",
                    model ="gpt-4o-mini",
                    #model ="gpt-3.5-turbo",
                    #model="gpt-3.5-turbo-instruct",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant that can analyze images."},
                        {"role":"user", "content": [
                            {
                              "type": "text",
                              "text": prompt
                            },
                            {
                              "type": "image_url",
                              "image_url": {
                                "url": f"data:image/jpeg;base64,{encoded_image}"
                              }
                            }
                        ]
                        }
                    ]
                )
                success = True
            except Exception as e:
                print(e)
                new_fac = fac/iFac
                print('new fac = ', new_fac)
                encoded_image = load_image(image_path,fac=new_fac)
                iFac += 1
        
        # Get the response from the API
        answer = response.choices[0].message.content
        # format answer
        answer_format = answer.replace("```json\n",'').replace("\n```",'')
        try:
            question_list['Response'] = json.loads(answer_format)
        except:
            question_list['Response'] = answer_format
            question_list['Error'] = 'JSON formatting'
        question_list['Response String'] = answer_format
        success = True
        #print('yes')
        #print('cant do this model')
        #print(e)
        #iFile += 1
        # new_fac = fac/iFac
        # print('new fac = ', new_fac)
        # encoded_image = load_image(image_path,fac=new_fac)
        # iFac += 1
            #continue

    # try to fix
    for level in qa:
        noErr = False
        if 'Error' in level:
            noErr = True
            try:
                iters = []
                for x in re.finditer(r'//(\s*)(.*)(\s*)\n', level['Response']):
                    #print(x)
                    iters.append(level['Response'][x.span()[0]:x.span()[1]])
                
                response = level['Response']
                for i in iters:
                    response = response.replace(i,'')
                level['Response'] = response
                del level['Error']
            except:
                noErr = False
                print('********')
                print('couldnt fix:', level['Response'])
                print('********')
                pass
    
        try:
            if type(level['Response']) != type({}):
                level['Response'] = json.loads(level['Response'])
        except:
            pass
    
        if not noErr and verbose:
            print('Q:', level['Q'])
            print('ChatGPT A:', level['Response'])
            print('Real A:   ', level['A'])
            print('')
            #if 'Error' in level:
            #    import sys; sys.exit()

    with open(dir_api + 'Picture'+str(int(iFile)) + '.pickle', 'wb') as ff:
        pickle.dump([qa], ff)

    iFile += 1

on 0 of 250
[Errno 2] No such file or directory: '/Users/jnaiman/Downloads/data_full_v2/Picture0.png'
on 1 of 250
have file already: /Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/Picture1.pickle
on 2 of 250
have file already: /Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/Picture2.pickle
on 3 of 250
[Errno 2] No such file or directory: '/Users/jnaiman/Downloads/data_full_v2/Picture3.png'
on 4 of 250
have file already: /Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/Picture4.pickle
on 5 of 250
have file already: /Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/Picture5.pickle
on 6 of 250
have file already: /Users/jnaiman/Dropbox/JCDL2024/paper_submission/chatgpt_api/Picture6.pickle
on 7 of 250
   on question: How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.
Error code: 400 - {'error': {'message': "This model's maxim

/opt/anaconda3/envs/FullProcess/lib/python3.10/site-packages/PIL/Image.py:3186: DecompressionBombWarning: Image size (134230125 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


   on question: How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.
Error code: 429 - {'error': {'message': 'Request too large for gpt-4o-mini in organization org-B3kc9FY70R5jhzuixVQ9GFqf on tokens per min (TPM): Limit 200000, Requested 860449. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
new fac =  0.5
   on question: How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.
Error code: 429 - {'error': {'message': 'Request too large for gpt-4o-mini in organization org-B3kc9FY70R5jhzuixVQ9GFqf on tokens per min (TPM): Limit 200000, Requested 860449. The input or output tokens must be reduced in order 

# ---- HERE ----

In [209]:
# get image
iFile = 1
image_path = '/Users/jnaiman/Downloads/data_full_v2/Picture'+str(int(iFile))+'.png'
encoded_image = load_image(image_path)

In [210]:
# get questions
with open('/Users/jnaiman/Downloads/data_full_v2/Picture'+str(int(iFile))+'_qa.json','r') as f:
    j = json.loads(f.read())
    j = json.loads(j)

In [212]:
qa = []

for k,v in j['Level 1']['Figure-level questions'].items():
    out = {'Q':v['Q'], 'A':v['A'], 'Level':'Level 1', 'type':'Figure-level questions', 'Response':""}
    qa.append(out)

# what kinds?
types = ['(words + list)', '(words)']

# get uniques
level_parse = 'Level 1'
plot_level = 'Plot-level questions'
qa = parse_qa(level_parse, plot_level, qa, j, types)

level_parse = 'Level 2'
plot_level = 'Plot-level questions'
qa = parse_qa(level_parse, plot_level, qa, j, types)

level_parse = 'Level 3'
plot_level = 'Plot-level questions'
qa = parse_qa(level_parse, plot_level, qa, j, types)


qa

[{'Q': 'How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.',
  'A': {'nrows': 1, 'ncols': 1},
  'Level': 'Level 1',
  'type': 'Figure-level questions',
  'Response': ''},
 {'Q': 'Assuming this is a figure made with matplotlib in Python, what is the plot style used?  Examples of plotting styles are "classic" or "ggplot". You are a helpful assistant, please format the output as a json as {"plot style":""} to store the matplotlib plotting style used in the figure.',
  'A': {'plot style': 'bmh'},
  'Level': 'Level 1',
  'type': 'Figure-level questions',
  'Response': ''},
 {'Q': 'Assuming this is a figure made with matplotlib in Python, what is the colormap that was used?  Examples of matplotlib colormaps are "rainbow" or "Reds". You are a helpful assistant, please format the output as a json as {"colormap":""} to store the matplotlib colormap used in the figure.',
  'A': {

In [213]:
#j['Level 2']['Plot-level questions']

In [214]:
#j['Level 1']['Plot-level questions']['nlines (plot numbers)'].keys()

In [215]:
#vv['A']

In [216]:
# setup
with open('/Users/jnaiman/.openai/key.txt','r') as f:
    api_key = f.read()

client = OpenAI(
  api_key=api_key.strip(),  # this is also the default, it can be omitted
)

responses = []

for question_list in qa:
    question = question_list['Q']
    # Prepare the API request
    prompt = f"I am going to show you an image. Here is the image: [Image: {encoded_image}]. Now, {question}"
    
    # Send the request to the GPT-4o API
    response = client.chat.completions.create(
        #model="gpt-4",
        #model="gpt-4o",
        model ="gpt-4o-mini",
        #model ="gpt-3.5-turbo",
        #model="gpt-3.5-turbo-instruct",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that can analyze images."},
            {"role":"user", "content": [
                {
                  "type": "text",
                  "text": prompt
                },
                {
                  "type": "image_url",
                  "image_url": {
                    "url": f"data:image/jpeg;base64,{encoded_image}"
                  }
                }
            ]
            }
        ]
    )
    
    # Get the response from the API
    answer = response.choices[0].message.content
    # format answer
    answer_format = answer.replace("```json\n",'').replace("\n```",'')
    try:
        question_list['Response'] = json.loads(answer_format)
    except:
        question_list['Response'] = answer_format
        question_list['Error'] = 'JSON formatting'
    question_list['Response String'] = answer_format

In [217]:
for level in qa:
    noErr = False
    if 'Error' in level:
        noErr = True
        try:
            iters = []
            for x in re.finditer(r'//(\s*)(.*)(\s*)\n', level['Response']):
                #print(x)
                iters.append(level['Response'][x.span()[0]:x.span()[1]])
            
            response = level['Response']
            for i in iters:
                response = response.replace(i,'')
            level['Response'] = response
            del level['Error']
        except:
            noErr = False
            print('********')
            print('couldnt fix:', level['Response'])
            print('********')
            pass

    try:
        if type(level['Response']) != type({}):
            level['Response'] = json.loads(level['Response'])
    except:
        pass

    if not noErr:
        print('Q:', level['Q'])
        print('ChatGPT A:', level['Response'])
        print('Real A:   ', level['A'])
        print('')
        #if 'Error' in level:
        #    import sys; sys.exit()

Q: How many panels are in this figure? You are a helpful assistant, please format the output as a json as {"nrows":"", "ncols":""} to store the number of rows and columns.
ChatGPT A: {'nrows': '1', 'ncols': '1'}
Real A:    {'nrows': 1, 'ncols': 1}

Q: Assuming this is a figure made with matplotlib in Python, what is the plot style used?  Examples of plotting styles are "classic" or "ggplot". You are a helpful assistant, please format the output as a json as {"plot style":""} to store the matplotlib plotting style used in the figure.
ChatGPT A: {'plot style': 'seaborn'}
Real A:    {'plot style': 'bmh'}

Q: Assuming this is a figure made with matplotlib in Python, what is the colormap that was used?  Examples of matplotlib colormaps are "rainbow" or "Reds". You are a helpful assistant, please format the output as a json as {"colormap":""} to store the matplotlib colormap used in the figure.
ChatGPT A: {'colormap': ''}
Real A:    {'colormap': 'gray'}

Q: What is the aspect ratio of this f